In [31]:
import numpy as np
import trimesh
import os

class mesh():

    def create_calculate_mesh_data(self,mesh_obj):

        with open(os.path.join('Temp','methanol'+'.face'),'r') as face_f:
            face = face_f.read()
        with open(os.path.join('Temp','methanol'+'.vert'),'r') as vert_f:
            vert = vert_f.read()

        mol_faces = np.vstack(np.char.split(face.split("\n")[0:-1]))[:, :3].astype(int) - 1
        mol_verts = np.vstack(np.char.split(vert.split("\n")[0:-1]))[:, :3].astype(np.float32)

        self.mol_mesh = trimesh.Trimesh(vertices=mol_verts, faces=mol_faces)

        self.mol_faces = self.mol_mesh.faces
        self.mol_verts = self.mol_mesh.vertices

        self.centroid = np.mean(self.mol_verts, axis=0)
        
        vertx = self.mol_verts[self.mol_faces]
        mol_areas = 0.5*np.linalg.norm(np.cross(vertx[:, 1, :] - vertx[:, 0, :], vertx[:, 2, :] - vertx[:, 0, :]), axis=1).astype(np.float32)
        self.mol_areas = mol_areas.reshape(-1,1)
        
        r = np.sqrt((self.mol_verts[:,0]-self.centroid[0])**2 + (self.mol_verts[:,1]-self.centroid[1])**2 + (self.mol_verts[:,2]-self.centroid[2])**2)
        self.R_mol = np.max(r)
        self.R_max_dist = np.max(np.sqrt((self.mol_verts[:,0])**2 + (self.mol_verts[:,1])**2 + (self.mol_verts[:,2])**2))
        
        element_vertices = self.mol_verts[self.mol_faces]
        self.mol_faces_centroid = np.mean(element_vertices, axis=1).astype(np.float32)

        self.mol_faces_normal = self.mol_mesh.face_normals
        self.mol_normal = self.mol_mesh.vertex_normals


R = 4
sphere = trimesh.creation.icosphere(radius=R, subdivisions=2)
mesh_obj = mesh()
mesh_obj.create_calculate_mesh_data(sphere)

In [40]:
import numpy as np
import trimesh
import plotly.graph_objs as go

# Create a trimesh mesh_obj
mesh_obj = trimesh.creation.icosphere(radius=2, subdivisions=2)

# Create an instance of the mesh class
mesh_obj = mesh()

# Calculate mesh data
mesh_obj.create_calculate_mesh_data(mesh_obj)

# Create a Plotly trace for the mesh
mesh_trace = go.Mesh3d(x=mesh_obj.mol_verts[:, 0], 
                       y=mesh_obj.mol_verts[:, 1], 
                       z=mesh_obj.mol_verts[:, 2], 
                       i=mesh_obj.mol_faces[:, 0], 
                       j=mesh_obj.mol_faces[:, 1], 
                       k=mesh_obj.mol_faces[:, 2],
                       color='lightblue', 
                       opacity=0.5)

# Create a trace for the vertex normals
vertex_normals_trace = go.Cone(x=mesh_obj.mol_faces_centroid[:, 0],
                                y=mesh_obj.mol_faces_centroid[:, 1],
                                z=mesh_obj.mol_faces_centroid[:, 2],
                                u=mesh_obj.mol_faces_normal[:, 0]*0.05,
                                v=mesh_obj.mol_faces_normal[:, 1]*0.05,
                                w=mesh_obj.mol_faces_normal[:, 2]*0.05,
                                sizemode="absolute",
                                sizeref=0.05,
                                showscale=False,
                                colorscale=[[0, 'red'], [1, 'red']],
                                hoverinfo='none')

edge_x = []
edge_y = []
edge_z = []

for element in mesh_obj.mol_faces:
    for i in range(3):
        edge_x.extend([mesh_obj.mol_verts[element[i % 3], 0], mesh_obj.mol_verts[element[(i + 1) % 3], 0], None])
        edge_y.extend([mesh_obj.mol_verts[element[i % 3], 1], mesh_obj.mol_verts[element[(i + 1) % 3], 1], None])
        edge_z.extend([mesh_obj.mol_verts[element[i % 3], 2], mesh_obj.mol_verts[element[(i + 1) % 3], 2], None])

edge_trace = go.Scatter3d(
    x=edge_x,
    y=edge_y,
    z=edge_z,
    mode='lines',
    line=dict(color='blue', width=2),
)

# Create the layout
layout = go.Layout(scene=dict(aspectmode="cube",
                               aspectratio=dict(x=1, y=1, z=1)))

# Create the figure
fig = go.Figure(data=[mesh_trace, vertex_normals_trace, edge_trace], layout=layout)

# Show the figure
fig.show()


In [33]:
import numpy as np
import trimesh
import plotly.graph_objs as go

# Create a trimesh mesh_obj
mesh_obj = trimesh.creation.icosphere(radius=2, subdivisions=2)

# Create an instance of the mesh class
mesh_obj = mesh()

# Calculate mesh data
mesh_obj.create_calculate_mesh_data(mesh_obj)

# Create a Plotly trace for the mesh
mesh_trace = go.Mesh3d(x=mesh_obj.mol_verts[:, 0], 
                       y=mesh_obj.mol_verts[:, 1], 
                       z=mesh_obj.mol_verts[:, 2], 
                       i=mesh_obj.mol_faces[:, 0], 
                       j=mesh_obj.mol_faces[:, 1], 
                       k=mesh_obj.mol_faces[:, 2],
                       color='lightblue', 
                       opacity=0.5)

# Create a trace for the vertex normals
vertex_normals_trace = go.Cone(x=mesh_obj.mol_verts[:, 0],
                                y=mesh_obj.mol_verts[:, 1],
                                z=mesh_obj.mol_verts[:, 2],
                                u=mesh_obj.mol_normal[:, 0]*0.1,
                                v=mesh_obj.mol_normal[:, 1]*0.1,
                                w=mesh_obj.mol_normal[:, 2]*0.1,
                                sizemode="absolute",
                                sizeref=0.05,
                                showscale=False,
                                colorscale=[[0, 'red'], [1, 'red']],
                                hoverinfo='none')

edge_x = []
edge_y = []
edge_z = []

for element in mesh_obj.mol_faces:
    for i in range(3):
        edge_x.extend([mesh_obj.mol_verts[element[i % 3], 0], mesh_obj.mol_verts[element[(i + 1) % 3], 0], None])
        edge_y.extend([mesh_obj.mol_verts[element[i % 3], 1], mesh_obj.mol_verts[element[(i + 1) % 3], 1], None])
        edge_z.extend([mesh_obj.mol_verts[element[i % 3], 2], mesh_obj.mol_verts[element[(i + 1) % 3], 2], None])

edge_trace = go.Scatter3d(
    x=edge_x,
    y=edge_y,
    z=edge_z,
    mode='lines',
    line=dict(color='blue', width=2),
)

# Create the layout
layout = go.Layout(scene=dict(aspectmode="cube",
                               aspectratio=dict(x=1, y=1, z=1)))

# Create the figure
fig = go.Figure(data=[mesh_trace, vertex_normals_trace, edge_trace], layout=layout)

# Show the figure
fig.show()


In [34]:
v_n = mesh_obj.mol_normal
f_n = mesh_obj.mol_faces_normal

In [35]:
np.sqrt(v_n[:,0]**2+v_n[:,1]**2+v_n[:,2]**2)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1.

In [36]:
np.sqrt(f_n[:,0]**2+f_n[:,1]**2+f_n[:,2]**2)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1.